### Name: James Bird
### Username: JXB1330
### ID number: 2212304
**please edit the above with your details to personlise this notebook**

# Laboratory 3a: Data Visualisation and Dimension Reduction

# Python for Engineers
(c) 2018-2021 Dr Neil Cooke, School of Engineering, Collaborative Teaching Laboratory, University of Birmingham

To do - work through solution, add some test code, maybe using in-built functions to validate.

# Introduction



In this lab, you will work towards animating an application of Principal Component Analysis (PCA). 

This is a technique very common in dimension reduction, and is a valuable tool in analysing a large set of data. 

You will also begin to handle external sets of data, which are be stored in .txt files. File handling is a key skill in Python and is one which you wil use often should you persue programming further.


# Learning Outcomes
In previous labs, you should have become more familiar and confident with programming in Python but you didn't handle data. However, Jupyter notebooks are designed primarily for this purpose - interactive computing - and data handling is a key skill.

By the end of this lab you will have:

> Become more comfortable with data handling and visualisation in interactive computing.

> appreciate the power of Principal compoenent analysis and how to implement it in Python.

**tasks you are required to do are highlighted in bold throughout this notebook**

## Tools to complete this lab

* Use this Jupyter notebook and complete/save your work in it. This requires a computing platform capable of running Python 3.x and Jupyter notebook (preferred). NOTE: pressing CTRL+Z on a highlighted cell will return it to itsprevious state. Use this to undo any changes you have made if you get lost and want to start again.

* If for whatever reason you do not or cannot use Jupyter notebooks, then use the PDF document of this as a 'lab sheet' and type / copy the python code into a py file in the order presented in the sheet, using the IDLE python editor and suitable comments between sections e.g. #Exercise 1. This will require a computing platform with Python 3.x installed. (not recommended)

# Dimension Reduction Overview

PCA is used in many applications in Electrical Engineering where multiple measurements are taken. For example, in control and power systems to make them more stable and spot faults, and communications systems for robustness and compression. For this exercise, you can primarily think of it as a technique for dimension (degree of freedom) reduction.  

## A simple MOCAP example

For example, take a motion capture application (MOCAP). A person is standing in front of a camera with 100 sensors around their body to capture their movements. The x-y coordinate of each point can be stored in its own 2 dimensional vector:

$$ v_{n} = \begin{bmatrix}x_{n},y_{n}\end{bmatrix} $$

To store the coordinates of every point on the body, the vectors can be written in a larger vector, V, where each element corresponds to the x-y coordinates (v) of a different point. For our example, this will be a 100-dimension vector:

$$ V = \begin{pmatrix} v_{0}\\ v_{1}\\ ...\\ v_{99} \end{pmatrix} = \begin{pmatrix} \begin{bmatrix}x_{0},y_{0}\end{bmatrix}\\ \begin{bmatrix}x_{1},y_{1}\end{bmatrix}\\ ...\\ \begin{bmatrix}x_{99},y_{99}\end{bmatrix} \end{pmatrix} $$

Now imagine the person moves. The position of each point of the body will change, so we can sample the position of the points for increments in time, t. The values of V for increasing t will be different, so we can write a matrix where each column represents a fixed increase in t. So visually, this will be a set of vectors V. This will produce a matrix with dimensions 100 x M, where M is the number of samples for each point's position i.e. the sample count. More generally, this will be a N x M matrix, where N is the number of points on the body and M is the sample count:

$$ \begin{pmatrix} V_{0} & V_{1} & ... & V_{M}\end{pmatrix} = \begin{pmatrix} v_{0,0} & v_{0,1} & v_{0,2} & v_{0,3} & v_{0,4} & v_{0,5} & ... & v_{0,M}\\ 
v_{1,0} & v_{1,1} & v_{1,2} & v_{1,3} & v_{1,4} & v_{1,5} & ... & v_{1,M}\\ 
v_{2,0} & v_{2,1} & v_{2,2} & v_{2,3} & v_{2,4} & v_{2,5} & ... & v_{2,M}\\ 
v_{3,0} & v_{3,1} & v_{3,2} & v_{3,3} & v_{3,4} & v_{3,5} & ... & v_{3,M} \\ 
v_{4,0} & v_{4,1} & v_{4,2} & v_{4,3} & v_{4,4} & v_{4,5} & ... & v_{4,M}\\ 
v_{5,0} & v_{5,1} & v_{5,2} & v_{5,3} & v_{5,4} & v_{5,5} & ... & v_{5,M}\\ 
... & ... & ... & ... & ... & ... & ... & ...\\
v_{N,0} & v_{N,1} & v_{N,2} & v_{N,3} & v_{N,4} & v_{N,5} & ... & v_{N,M}
\end{pmatrix} $$

This matrix is large. But what if there were 1000 points on the body? Or if there was an application which measured even more points than that? 

We need to reduce the number of points to make the data easier to handle. We call this dimension reduction.

## Why does reducing the number of points make data the easier to handle? 

Firstly, there is the smaller memory footprint for storage and resulting lower bandwidth for communication. 

As importantly, handling the data involves extracting/inferring some information e.g. what activity is being performed in the mocap data. This requires looking for patterns in the data e.g. recurring and similar (but not exactly the same) values. The lower the number of dimensions (called more precisely Degrees of Freedom or DoF), the easier these patterns are to spot e.g. less data is needed. This idea of too many dimensions to spot patterns is called "The curse of dimensionality".


## How do we compute dimension reduction? 

The simplest method is feature selection - discard some points and keep others. The J most important elements of V could be identified, and the remaining elements are removed. In this example, J is the number of points on the body that move the most. You can imagine that the hand will move a lot more than a point on the forearm, so that positions of the forearm will be removed from the matrix. This gives us a dataset that is easier to work with.

This is an example of feature selection - discarding those features/points/elements which do not move much i.e. have a small variance. 

We can set J to be any value we like, so if we only want 10 points to define our body, we can find the 10 most important points in the vector V. This will reduce the vector by 90 dimensions, while still preserving the overall structure of the data:

$$ \begin{pmatrix}v_{0}\\ v_{1}\\ v_{2}\\ v_{3}\\ v_{4}\\ v_{5}\\ v_{6}\\v_{7}\\ v_{8}\\ v_{9}\\ v_{10}\\ v_{11}\\ v_{12}\\ ...\\ v_{99}\end{pmatrix}------------->\begin{pmatrix}v_{0}\\ v_{1}\\ v_{2}\\ v_{3}\\ v_{4}\\ v_{5}\\ v_{6}\\v_{7}\\ v_{8}\\ v_{9}\\ \end{pmatrix} $$

## How do you determine the most important points/features? 

In the example above we looked at the greatest movement which could be measured using a summary statistic e.g. variance, range and so on.

PCA uses variance as its "measure of importance", but does not simply deselect points from the origial data if they have small variances. Instead, we transform our original data to a new feature space using an eignevalue decomposition of the covariance matrix, so the variance of points/features in the new space can be maximised with respect to as fewer points as possible.




# Principal Component Analysis

## PCA visualisation


It is easy to visualise PCA if we consider dimensions as we see the world - 3D, with relative positions of points measured by pont-to-point distances.

Even easier, consider the MOCAP example given earlier and simplify it so we are collecting only 2D from one point to the body - an x and y coordinate to indicate the persons position as they walk along (we assume height - the 3rd dimension - z coordinate - is constant). 

The graphs below show the movement - there are 4 graphs.

Leftmost graph: The camera is overhead looking down on the person. There is a postive correlation between x and y so the person is moving diagonally in relation to the camera.

Rightmost graph: Although our data is only 2 dimensional, we wish to reduce it to one dimension to efficiently capture the motion (it turns out the person is walking along a straight path so we have no need to store their transgressions from the path).

<img src="PCA_image.PNG">

Middle two graphs: To simplify the data, we will apply PCA to represent the 2-dimensional relationship as a 1-dimensional line. Here the dimension reduction is from 2D to 1D. The intuitive step is to rotate the axes relative to the data so that there is axis capturing the largest variance. We call this a "principal component" (this is the horizontal axis in the figure). We may then discard the vertical axis making data handling easier. Note the axes always remain orthogonal to once another e.g. they contain unique information. 

In higher dimensional problems e.g. 100 DoF to 5 DoF, the process is exactly the same although our intuititon can break down since we cannot visualise this in our limited 3D vision...

## Computational steps

Computationally, the PCA step requires 4 computations. For our 2D example:

> Find the mean of the x coordinates and of the y coordinates..

> Subtract the mean x value from the x coordinates and the mean y value from the y coordinates..

> Find a new set of axes so that the x values and y values to have maximum variance in a princpal component...

> Reduce the dimension of the data by eliminating non-principal component...


You are now ready to implement PCA in python...





# Exercise 1: Load Data

The dataset containing the x-y coordinates for this lab are stored in a text file called 'coords.txt' which you shold put in the same directory as this notebook. 

**Write a function called 'get_coords' which takes the argument 'filename' and opens the file for reading. **

Once the file is open, use a for loop to iterate through each line of the file and print the line to the console. Call the function using the argument 'coords.txt'. You should use the function open() and close() when doing this.

In [123]:
# Define function

def get_coords(filename): #ad arguments
    filename = open(filename,'r')
    for line in filename:
        print(line)
    filename.close()    
    return 0 #<something>

get_coords('coords.txt') #call function

1.9,2.2

8.7,9.3

7.6,8.1

2.1,1.7

3.9,4.1

6.2,5.7

5.3,4.7

4.5,5.2

8.7,7.8

3.0,4.3

9.7,8.6

7.2,7.5


0

# Exercise 2: Parse data using .split() function

**Modify the program above to split each line at the comma, and print the elements of the list to the console (not the list itself).**

Hint: Using indexing will help here

In [221]:
# Define function

def get_coords(filename): #ad arguments
    filename = open(filename,'r')
    for line in filename:
        print(*line.split(','), end = '')
    filename.close()    
    return 0 #<something>

get_coords('coords.txt') #call function

1.9 2.2
8.7 9.3
7.6 8.1
2.1 1.7
3.9 4.1
6.2 5.7
5.3 4.7
4.5 5.2
8.7 7.8
3.0 4.3
9.7 8.6
7.2 7.5

0

# Exercise 3: Structure data as an array of floats

**Modify the program again to append the coordinates (as floats) to a list called 'coords'. Make sure each set of x-y coordinates are stored in their own list, so coords is a list of lists. Return the list at the end of the function.**

Hint: Remember the casting (first seen in lab 1)

In [196]:
# Modify function
def get_coords_list(filename):
    filename = open(filename,'r')
    for line in filename:
        coords.append(line.splitlines())

    return x

coords = get_coords_list('coords.txt')
coords # output to console

AttributeError: 'NoneType' object has no attribute 'append'

# Exercise 4: Compute Summary Statistic (Calculate Means)

A data point is an x and y coordinate e.g. it has 2 dimensions. There are multiple points e.g. multiple x's and y's. Therefore on each dimension we can describe it by a summary statistic.

The next function is the first step of a PCA: to find the mean of each dimension e.g. x and y. 

**Write a function 'get_mean' with input argument being the list 'coords'. Create a loop which cycles through the list 'coords' and adds up all the values, then divides by the number of elements in the list. Store the average  and average y values in a list and return this from the function.**

In [28]:
# Define function
def get_mean(coords)
    return

get_mean(coords) #call the function

SyntaxError: invalid syntax (576231661.py, line 2)

In [29]:
#you can use the test code below to test your code against a library version (numpy)
import numpy as np
import unittest
class MyTest(unittest.TestCase):
    def test(self):
        self.assertEqual(get_mean(coords),np.mean(coords,axis=0).tolist()) #duplicate this line with different values for multiple testing
res = unittest.main(argv=[''], verbosity=3, exit=False)
assert len(res.result.failures) == 0

test (__main__.MyTest) ... ERROR

ERROR: test (__main__.MyTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/6q/475tbh2d161gpk8131qq47pm0000gn/T/ipykernel_82664/1260024911.py", line 6, in test
    self.assertEqual(get_mean(coords),np.mean(coords,axis=0).tolist()) #duplicate this line with different values for multiple testing
NameError: name 'get_mean' is not defined

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


**now extend your function get_mean() to accept coordinates of any number of dimensions (not just 2)**

In [30]:
#accept a list of coordinates on any number of dimensions NxM
def get_mean_general(coords):
    return 0

coords_random = [[1,2,3,4],[2,3,4,5] #Set-up a random 2D array dimensions NxM
mean = get_mean_general(coords_random)
print(mean)

SyntaxError: invalid syntax (3745559359.py, line 6)

In [31]:
#you can use the test code below to test your code against a numpy module mean function
import numpy as np
import random as r
import unittest

N = r.randint(1,100)
M = r.randint(1,100)
coords_random = [[r.random()]*N]*M
get_mean_general(coords_random)

class MyTest(unittest.TestCase):
    def test(self):
        self.assertEqual(get_mean_general(coords_extended),np.mean(coords_extended,axis=0).tolist()) #duplicate this line with different values for multiple testing
res = unittest.main(argv=[''], verbosity=3, exit=False)
assert len(res.result.failures) == 00

NameError: name 'get_mean_general' is not defined

# Exercise 5: Normalise Data to origin (Subtract the Means)

The next step for PCA is to shift the data so that is centred about the origin e.g. the mean of x and the mean of y are both equal to 0.0.

**write a function 'get_cmm' (cmm = coordinates minus mean) which creates a new set of coordinates, equal to the initial set, with the x mean subtracted from the x coordinate and the y mean subtracted from the y coordinates.**

HINT:The function should take in two arguments.
Store the new coordinates in a list called 'cmm'.
There are two ways to do this, see which ways you can come up with.

Return the list, 'cmm', from the function.

In [ ]:
# Define function
def get_cmm(coords,mean):
    return 0

#test function
get_cmm(coords,mean)


In [ ]:
#you can use the test code below to test your function compared to numpy
import unittest
class MyTest(unittest.TestCase):
    def test(self):
        self.assertEqual(get_cmm(coords,mean),[[-3.8333333333333335, -3.5666666666666655],
 [2.966666666666666, 3.533333333333335],
 [1.8666666666666663, 2.333333333333334],
 [-3.6333333333333333, -4.0666666666666655],
 [-1.8333333333333335, -1.666666666666666],
 [0.4666666666666668, -0.06666666666666554],
 [-0.43333333333333357, -1.0666666666666655],
 [-1.2333333333333334, -0.5666666666666655],
 [2.966666666666666, 2.033333333333334],
 [-2.7333333333333334, -1.466666666666666],
 [3.966666666666666, 2.833333333333334],
 [1.4666666666666668, 1.7333333333333343]]) #duplicate this line with different values for multiple testing
res = unittest.main(argv=[''], verbosity=3, exit=False)
assert len(res.result.failures) == 0

# Exercise 6: Compute the Covariance Matrix

Now we have some mean substracted data centred on the origin, we can compute the summary statsitic used by PCA to help reduce dimensions - the covariance matrix. 

This function will tell us how much vairation is in each of x and y, and also how they vary together (as described in more detail below).

The covariance function is one of three functions you need to implement.

It will take a N x 2 matrix (in our case the list 'cmm') and produce a 2 x 2 matrix. 

The covariance matrix takes the following form:

\begin{pmatrix}
A & B\\ 
B & C
\end{pmatrix}

You can see that the elements in the top right and bottom left positions have the same value; the matrix is symmetric. All the elements are real; the matrix is thus a 'real symmetric' matrix.

A denotes the variance of the data in the x direction. The larger the value of A, the greater the spread of the data in x.

C denotes the variance of the data in the y direction. The larger the value of C, the greater the spread of the data in y.

B denotes the covariance. This is how much x and y depend on each other. Large positive values indicate strong positive correlation e.g. as x increases, so would y. Likewise, large negative values indicate strong negative correlation e.g. as x increases, y decreases. Values close to zero indicate the x and y values have little relation at all, e.g. if x increases y may either decrease or increase, so x and y are uncorrelated (as would be the case if the value of B tends towards zero).

The covariance matrix can be found by a simple formula:

$$ M = \frac{1}{N-1} \cdot X\cdot X^{T} $$

Where:

> M is the covariance matrix

> N is the number of data points (a pair of x-y coordinates make 1 data point)

> X is the N x 2 list containing the x-y coordinates

> X^T is the 2 x N transpose of X

**Define a function called 'get_covariance_matrix'. Use a for loop to multiply 'cmm' by 'cmm' transposed (remember to divide each element of the new matrix by (N-1). This will give you the values of A, B and C. Create a new list called 'covariance_matrix' which stores the lists [a,b] and [b,c] to represent the covariance matrix M.**

Hint: Think about how to get the value of N, this is the total number of x-y pairs in 'cmm'.

In [ ]:
def get_covariance_matrix(cmm):
    #
    #
    # Write function here
    #
    #
    #
    return 0

cov = get_covariance_matrix(cmm)
print(cmm,cov)

# Exercise 7: Finding Unit Vectors using Numpy module

The next step in the process is to find the eigenvalues and eigenvectors of the covariance matrix M. 

This will give us the direction of maximum variation of the data. 

Essentially, the maximum variation of positively correlated data lies along the diagonal line the data forms. 

This direction will be approximated by a straight line which is an eigen vector of the covariance matrix. 

As M is a 2 x 2 matrix, there will be a second eigen vector. 

This is the vector perpendicular to the first, and represents the second biggest variation in the data.

These two vectors are perpendicular, and will align the data along a single axis. 

For this reason, these vectors will form our new coordinate system (bases).

Another way to think about this is that we have created a new 2D coordinate system from our existing 2D coordinate system by rotating the axis of our origin-centred x and y data so that the biggest viaration in data is along the first i.e. the path where the person is walking.

------------------------------------------------------------------------------------------------------------------------------

## numpy module

Python has a module named 'numpy' which calculates the eigenvalues and eigenvectors for us. These have been stored in the variables eigen_values and u respectively. Both of these return lists: eigen_values stores a list of 2 elements, u stores two lists of 2 elements.

Before we can use the eigenvectors as a coordinate system, we must first normalise them. To do this, define a function below get_covariance_matrix called get_unit_vector which takes the variable 'u' as an argument.

Find the length of each vector and divide the elements in that vector by its length. Store the new elements back in 'u' (to overwrite it) and return 'u' back to the function.

Reorganise the elements of 'u' so that the normalised vectors no longer lie horizontally, but vertically. This is an important step for PCA. See the diagram below:

$$ \begin{pmatrix}u_{1} & u_{2}\\ v_{1} & v_{2}\end{pmatrix} -> \begin{pmatrix}u_{1} & v_{1}\\ u_{2} &v_{2}\end{pmatrix} $$

In [1]:
import numpy as np
    
def get_unit_vector(u):
    #
    # Find the unit vector
    #
    # Reorganise elements of u, HINT: use tuple assignment
    
    return u

get_unit_vector(cov)

NameError: name 'cov' is not defined

# Exercise 8: Display Calculated Values

**Modify the get_covariance_matrix function to print out the values of:**

> **The covariance matrix**

> **The eigen values**

> **The unit vectors**

with all values rounded to 2 decimal places.

In [ ]:
import numpy as np

def get_covariance_matrix(cmm):
    #
    #
    # Write function here
    #
    #
    #
    eigen_values, u = np.eig(covariance_matrix)
    u = get_unit_vector(u)
    #
    # Add print statements
    #
    return covariance_matrix, u
    
def get_unit_vector(u):
    #
    # Find the unit vector
    #
    # Reorganise elements of u, HINT: use tuple assignment
    
    return u

# Exercise 9: Change the Coordinate System

The new matrix we have defined (which is is made up of normalised eigen vectors in each column) will transform each datapoint into the new vector space. This means that when we graph the data, there will no longer be a diagonal line, but a horizontal one. This will allow us to perform dimension reduction later on, but at this stage in the program, will simple make the data easier to understand.

**Define a function called 'change_bases' which takes 'u' and 'cmm' as arguments. Computer the pre-multiplication of u onto cmm, and store the new coordinates in a list called 'new_data'. The data should take the form of a list of list, with each element being a two-dimensional list storing the x and y (techincally u and v) coordinates of the new data points.**

In [ ]:
def change_bases(u, cmm):
    new_data = [] # this is the data after it has changed bases
    
    for count in range(len(cmm)):
        # Pre-multiply cmm by u
        
        # Store new values in new_data
        
    return new_data

# Exercise 10: Reduce the Dimensions (and speed things up)

Now the coordinate system has been changed, the data will roughly be a flat horizontal line along the x axis (technically the u axis as the coordinate system has been changed from x-y to a new system, typically called u-v). To reduce the dimensions in this example, we are going to remove the y (v) coordinates as these dont vary much and little information about the system will be lost when removing them. The same sense of the data is achieved with this approximation, and as the dataset is smaller, the data is easier to work with.

**For this exercise the function has been written for you using iteration - a for loop. Rewrite the function so that it uses a more efficient list comprehension instead.**

In [ ]:
def dimension_reduction(new_data):
    #reduced_data = []
    #for count in range(len(new_data)):
    #    reduced_data.append([new_data[count][0],0])
    return reduced_data

# Exercise 11: Output the Reduced Dimensionality data to a File

**Write a function which takes in two parameters: the array of reduced data and the name (as a string) of the file where the data should be saved.**

**Using a for loop, create a variable called 'text_data' which stores every u-v coordinate, separate by a comma, with a new line after each u-v pair.**

**Write the data to the file and remember to close the file when the writing is complete.**

In [ ]:
def write_reduced_data_to_file(#parameter_1, parameter_2):
    text_data = ''
    # for loop
        # Add strings to text_data
    # Write text data to file
    # Close file

# Exercise 12: Visualise the Data

Up to now you have been working "blind" with the data, in order for you to concentrate on numerical processing. However, when working with data in interactive computing applications, you will frequently want to visualise what you are doing.

To visualise this data, a module called MatPlotLib is used. It is reccommended extra reading for those interesting in using Python for mathematical and graphic-based programs.

**To complete this lab, make sure you have run completed and run all of the cells from exercises 4, 5, 8, 9, 10 and 11. **

**Once you have done completed this, simply run the cell below to see your final result. This is the analysis and dimensional reduction of a 2D dataset into a single 1D line.**


In [169]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy.linalg as np
from math import sqrt

mpl.rcParams.update({'font.size': 22})
    
    
def plot_graphs(coords, cmm, new_data, reduced_data,delay=0.5):
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(1,1,1)
    ax.labelsize : 24
    ax.set_xlim([-10,10]); ax.set_ylim([-10,10])
    plt.ion()
    #plt.tight_layout()
    while 1:
        ax.set_xlim([-10,10]); ax.set_ylim([-10,10])
        ax.set_title('Raw correlated data')
        ax.scatter(*zip(*coords), color = 'blue')
        fig.canvas.draw()

        plt.pause(delay)
        ax.clear()

        ax.set_xlim([-10,10]); ax.set_ylim([-10,10])
        ax.set_title('Subtract the mean')
        ax.scatter(*zip(*cmm), color = 'red')
        fig.canvas.draw()

        plt.pause(delay)
        ax.clear()

        ax.set_xlim([-10,10]); ax.set_ylim([-10,10])
        ax.set_title('Change the axes')
        ax.scatter(*zip(*new_data), color = 'red')
        fig.canvas.draw()

        plt.pause(delay)
        ax.clear()
        
        ax.set_xlim([-10,10]); ax.set_ylim([-10,10])
        ax.set_title('Remove the y dimension')
        ax.scatter(*zip(*reduced_data), color = 'red')
        fig.canvas.draw()
        
        plt.pause(delay)
        ax.clear()
        

def main():
    coords = get_coords('coords.txt')
    mean = get_mean(coords)
    cmm = get_cmm(coords, mean) #cmm is the coords_minus_mean, this gives 'zero-mean' data
    covariance_matrix, u = get_covariance_matrix(cmm)
    new_data = change_bases(u, cmm)
    reduced_data = dimension_reduction(new_data)
    write_reduced_data_to_file(reduced_data,'new_data.txt')
    plot_graphs(coords, cmm, new_data, reduced_data, 1)
    
main()

1.9 2.2
8.7 9.3
7.6 8.1
2.1 1.7
3.9 4.1
6.2 5.7
5.3 4.7
4.5 5.2
8.7 7.8
3.0 4.3
9.7 8.6
7.2 7.5

NameError: name 'get_mean' is not defined

# final note

Congratulations - you have demonstrated basic data handling techniques in Python!

There are a couple of useful modules and function that you could use to improve your code: sklearn, scipy and pandas for data handling.

In reality, PCA is used for much higher dimensions than the "toy" problem here;  to simplify 100s of dimensions down to the order of 10.

# End of Lab

This is the end of the Lab. You should now have a handle on data handling, use of numpy and visualisation.

Jupyter notebooks are not just for coding; you can write in them too in markup language. So note in the next cell what what you learned/found difficult or easy in the space below.




**please reflect on your learning below**

### I rate this lab (out of 5): 5

### What I find easy:


### What I find difficult:


### What I should improve:


# Next Learning Steps

* **Upload this completed ipynb notebook to canvas**


# Python for Engineers
(c) 2018,2021 Dr Neil Cooke, School of Engineering, Collaborative Teaching Laboratory, University of Birmingham